In [1]:
from dotenv import load_dotenv
from langgraph.graph.message import add_messages
from langgraph.graph import START, END, StateGraph
from langgraph.checkpoint.memory import MemorySaver
from typing_extensions import Annotated
from typing import TypedDict
from langchain.chat_models import init_chat_model

load_dotenv()


True

In [2]:
class State(TypedDict):
    messages: Annotated[list , add_messages]

graphBuilder = StateGraph(State)

In [3]:
model = init_chat_model(model="deepseek/deepseek-chat-v3-0324:free" , model_provider="openai")

def llm(state: State):
    return {"messages" : model.invoke(state["messages"])}

graphBuilder.add_node("llm" , llm)

graphBuilder.add_edge(START , "llm")
graphBuilder.add_edge("llm", END)

In [4]:
checkpointer = MemorySaver()
graph = graphBuilder.compile(checkpointer=checkpointer)

In [ ]:
config = {"configurable" : {"thread_id" : "5"}}

while True:
    user_input = input("User: ")

    if user_input == 'q':
        break

    print(
        graph.invoke({
                "messages" : 
                    {
                        "role": "user",
                        "content": user_input
                    },
                },config=config
            )["messages"][-1], flush=True
        )

The above example shows the use of checkpointer for memory retention for basic to and for converstaion between a chatbot and a human.

The below is a more intuitive and easy way of undersanding the more complex and intriquite workings of this feature

In [20]:
# Get state : get the latest state snapshot

graph.get_state(config)

StateSnapshot(values={'messages': [HumanMessage(content='hello my name is raghav', additional_kwargs={}, response_metadata={}, id='02c44356-da72-41d1-99f8-8a9a7a46732b'), AIMessage(content='Hello, Raghav! Nice to meet you. 😊 How can I assist you today? Let me know if you have any questions or need help with anything.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 10, 'total_tokens': 44, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek/deepseek-chat-v3-0324', 'system_fingerprint': None, 'id': 'gen-1744109817-A69duf5S5TACdhW0djD3', 'finish_reason': 'stop', 'logprobs': None}, id='run-5f4fdec5-39fc-4cf4-ad90-57644cca2dbb-0', usage_metadata={'input_tokens': 10, 'output_tokens': 34, 'total_tokens': 44, 'input_token_details': {}, 'output_token_details': {}}), HumanMessage(content='what are you doing currently', additional_kwargs={}, response_metadata={}, id='de2b0b85-3faf-4964-a59e-c

In [ ]:
# Get state history: Get teh full of the graph execution for a given thread 

config1 = {"configurable" : {"thread_id" : "1"}}
print(list(graph.get_state_history(config)))

[StateSnapshot(values={'messages': [HumanMessage(content='hello my name is raghav', additional_kwargs={}, response_metadata={}, id='02c44356-da72-41d1-99f8-8a9a7a46732b'), AIMessage(content='Hello, Raghav! Nice to meet you. 😊 How can I assist you today? Let me know if you have any questions or need help with anything.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 10, 'total_tokens': 44, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek/deepseek-chat-v3-0324', 'system_fingerprint': None, 'id': 'gen-1744109817-A69duf5S5TACdhW0djD3', 'finish_reason': 'stop', 'logprobs': None}, id='run-5f4fdec5-39fc-4cf4-ad90-57644cca2dbb-0', usage_metadata={'input_tokens': 10, 'output_tokens': 34, 'total_tokens': 44, 'input_token_details': {}, 'output_token_details': {}}), HumanMessage(content='what are you doing currently', additional_kwargs={}, response_metadata={}, id='de2b0b85-3faf-4964-a59e-